In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('./data/gemstone.csv')
df.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [3]:
x=df.drop(labels=['price'],axis=1)
y=df[['price']]
x.head()

,id,carat,cut,color,clarity,depth,table,x,y,z
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77


In [4]:
num_col=x.columns[x.dtypes!='object']
num_col

Index(['id', 'carat', 'depth', 'table', 'x', 'y', 'z'], dtype='object')

In [5]:
cat_col=x.columns[x.dtypes=='object']
cat_col

Index(['cut', 'color', 'clarity'], dtype='object')

In [6]:
cut_cat=['Ideal','Premium' ,'Very Good' ,'Good' ,'Fair']
color_cat = ['D','E','F','G','H','I','J']
clarity_cat = ['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

In [8]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer


In [9]:
num_pipeline =Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='median')),
        ('scaler',StandardScaler())
    ]
)

cat_pipeline = Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='most_frequent')),
        ('ordinalEncoder',OrdinalEncoder(categories=[cut_cat,color_cat,clarity_cat])),
        ('scaler',StandardScaler())
    ]
)

In [10]:
preprocessor=ColumnTransformer([
    ('num_pipeline',num_pipeline,num_col),
    ('cat_pipeline',cat_pipeline,cat_col)
])

In [11]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=30)

In [12]:
x_train=pd.DataFrame(preprocessor.fit_transform(x_train),columns=preprocessor.get_feature_names_out())
x_test=pd.DataFrame(preprocessor.transform(x_test),columns=preprocessor.get_feature_names_out())
x_train.head()

,num_pipeline__id,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,1.274919,-0.975439,-0.849607,-0.121531,-1.042757,-1.080970,-1.123150,-0.874076,1.528722,1.352731
1,-1.102907,0.235195,1.833637,-0.121531,0.318447,0.279859,0.485354,2.144558,-0.935071,-0.646786
2,-0.997613,0.494617,0.815855,0.399800,0.570855,0.606458,0.673737,0.132136,0.296826,0.686225
3,-1.173764,-1.018676,0.260701,0.921131,-1.214034,-1.244270,-1.195605,0.132136,0.296826,0.019720
4,-0.914812,-0.953821,-0.664555,-0.642862,-1.069801,-1.044681,-1.094168,-0.874076,2.144670,1.352731


In [14]:
x_train=x_train.drop(labels=['num_pipeline__id'],axis=1)
x_test=x_test.drop(labels=['num_pipeline__id'],axis=1)

In [15]:
x_train.head()

,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,-0.975439,-0.849607,-0.121531,-1.042757,-1.080970,-1.123150,-0.874076,1.528722,1.352731
1,0.235195,1.833637,-0.121531,0.318447,0.279859,0.485354,2.144558,-0.935071,-0.646786
2,0.494617,0.815855,0.399800,0.570855,0.606458,0.673737,0.132136,0.296826,0.686225
3,-1.018676,0.260701,0.921131,-1.214034,-1.244270,-1.195605,0.132136,0.296826,0.019720
4,-0.953821,-0.664555,-0.642862,-1.069801,-1.044681,-1.094168,-0.874076,2.144670,1.352731


In [16]:
from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [17]:
import numpy as np
def evalute_model(true,predicted):
    mae=mean_absolute_error(true,predicted)
    mse=mean_squared_error(true,predicted)
    rmse=np.sqrt(mse)
    r2_sq=r2_score(true,predicted)
    return mae, mse, r2_sq

In [19]:
models={
    'LinearRegression': LinearRegression(),
    'Lasso': Lasso(),
    'Ridge':Ridge(),
    'ElasticNet':ElasticNet()

}
trained_model_list=[]
model_list=[]
r2_list=[]
for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(x_train,y_train)
    y_pred=model.predict(x_test)
    mae ,rmse, r2_square=evalute_model(y_test,y_pred)
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    
    print('model training performance')
    print("Rmse ",rmse)
    print("mae ",mae)
    print("r2_score ",r2_square*100)
    r2_list.append(r2_square)
    print('='*35)
    print('\n')


LinearRegression
model training performance
Rmse  1028002.7598132559
mae  674.0255115796832
r2_score  93.68908248567512


Lasso
model training performance
Rmse  1027949.4559693959
mae  675.0716923362162
r2_score  93.68940971841704


Ridge
model training performance
Rmse  1028005.2293677676
mae  674.0555800798198
r2_score  93.68906732505937


ElasticNet
model training performance
Rmse  2351365.3822896425
mae  1060.7368759154729
r2_score  85.56494831165182


